<a href="https://colab.research.google.com/github/AtreyeeBhattacharjee/Text-Classification-using-Tensorflow/blob/master/AB_Text_Classification_using_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing TensorFlow, TensorFlow Hub and Datasets**

In [ ]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow_datasets

# **Importing Libraries**

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# **Downloading IMDB Dataset**

In [ ]:
#The training set splitted into 60% and 40% to end up with 15,000 examples for training,
#10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",
                                            split=('train[:60%]', 'train[60%:]', 'test'),
                                            as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete39SIEI/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete39SIEI/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete39SIEI/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


***Exploring dataset***

In [ ]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [ ]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch #gives reviews : positive review as 1 and negative as 0

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

# **Text Embedding**

In [ ]:
#we will fetch a pre-trained model from tensorflow_hub
hub_layer = hub.KerasLayer("https://kaggle.com/models/google/gnews-swivel/frameworks/TensorFlow2/variations/tf2-preview-20dim/versions/1",
                           output_shape=[20],
                           input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

# **Creation of Sequential Model**

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu')) #16 layers in Density layer with relu activation function
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_2 (Dense)             (None, 16)                336       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# **Loss function & Optimizer**

In [ ]:
model.compile(optimizer='adam', loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# **Model Training, Fitting & Testing**

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(100), epochs=25,
                    validation_data = validation_data.batch(100), verbose=1)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


150/150 [==============================] - 7s 42ms/step - loss: 0.7449 - accuracy: 0.5505 - val_loss: 0.6342 - val_accuracy: 0.6572
Epoch 2/25
150/150 [==============================] - 4s 24ms/step - loss: 0.5948 - accuracy: 0.6990 - val_loss: 0.5596 - val_accuracy: 0.7352
Epoch 3/25
150/150 [==============================] - 4s 24ms/step - loss: 0.4974 - accuracy: 0.7815 - val_loss: 0.4496 - val_accuracy: 0.8042
Epoch 4/25
150/150 [==============================] - 4s 24ms/step - loss: 0.3752 - accuracy: 0.8468 - val_loss: 0.3699 - val_accuracy: 0.8422
Epoch 5/25
150/150 [==============================] - 4s 25ms/step - loss: 0.3006 - accuracy: 0.8817 - val_loss: 0.3374 - val_accuracy: 0.8562
Epoch 6/25
150/150 [==============================] - 4s 27ms/step - loss: 0.2552 - accuracy: 0.9009 - val_loss: 0.3152 - val_accuracy: 0.8664
Epoch 7/25
150/150 [==============================] - 4s 24ms/step - loss: 0.2201 - accuracy: 0.9166 - val_loss: 0.3027 - val_accuracy: 0.8736
Epoch 8/25

# **Model Evaluation**

In [ ]:
results = model.evaluate(test_data.batch(100), verbose=2)
results

250/250 - 2s - loss: 0.6342 - accuracy: 0.8420 - 2s/epoch - 9ms/step


[0.634192943572998, 0.8420000076293945]